In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,431 kB]
Get:13 http:

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [6]:
# Create table where all rows of total_votes count is equal to or greater than 20
total_votes_filtered = df.filter("total_votes>=20")
total_votes_filtered.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [7]:
# create a new DataFrame or table to retrieve all the rows where the number of
# helpful_votes divided by total_votes is equal to or greater than 50%.

helpful_votes = total_votes_filtered.filter("helpful_votes/total_votes>=0.5")
helpful_votes.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [8]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame
# that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

vine_paid = helpful_votes.filter(helpful_votes.vine == 'Y')
vine_paid.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44759540|R3A71VR1JZD8WF|B00JGWN8O8|     378897034|Petcube Interacti...|    Pet Products|          2|           27|         30|   Y|                N|Great idea, not s...|I do not hate thi...| 2015-08-29|
|         US|   37388532|R16OMUJIGI18JZ|B00IP05CUA|     833937853|Wellness Kittles ...|    Pet Products|          5|    

In [9]:
# Repeat Step 3, but this time retrieve all the rows where the review 
# was not part of the Vine program (unpaid), vine == 'N'.

vine_unpaid = helpful_votes.filter(helpful_votes.vine == 'N')
vine_unpaid.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [10]:
#PAID VINE
# Total number of reviews 
total_reviews = vine_paid.count()
total_reviews

#total 5 star reviews
five_star_reviews_paid = vine_paid.filter("star_rating = 5").count()
five_star_reviews_paid

# percentage of 5 star reviews
percentage_five_star = five_star_reviews_paid/total_reviews
percentage_five_star

0.38235294117647056

In [11]:
paid_vine_summary = spark.createDataFrame([
                                   (total_reviews, five_star_reviews_paid, percentage_five_star),
], ["Total Reviews", "5 Star Reviews", "% of 5 Star Reviews"])
paid_vine_summary.show()

+-------------+--------------+-------------------+
|Total Reviews|5 Star Reviews|% of 5 Star Reviews|
+-------------+--------------+-------------------+
|          170|            65|0.38235294117647056|
+-------------+--------------+-------------------+



In [12]:
#UNPAID VINE
# Total number of reviews 
total_reviews_unpaid = vine_unpaid.count()

#total 5 star reviews
five_star_reviews_unpaid = vine_unpaid.filter("star_rating = 5").count()


# percentage of 5 star reviews
percentage_five_star_unpaid = five_star_reviews_unpaid/total_reviews_unpaid


In [13]:
unpaid_vine_summary = spark.createDataFrame([
                                   (total_reviews_unpaid, five_star_reviews_unpaid, percentage_five_star_unpaid),
], ["Total Reviews", "5 Star Reviews", "% of 5 Star Reviews"])
unpaid_vine_summary.show()

+-------------+--------------+-------------------+
|Total Reviews|5 Star Reviews|% of 5 Star Reviews|
+-------------+--------------+-------------------+
|        37840|         20612| 0.5447145877378435|
+-------------+--------------+-------------------+

